#EECE798S - Assignment 5

###Mohamad Mazeh

In [1]:
!pip install -q openai openai-agents requests typing_extensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.4 which is incompatible.


In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [3]:
import os
os.makedirs("map_servers", exist_ok=True)
os.makedirs("tests", exist_ok=True)

with open("map_servers/__init__.py", "w") as f:
    f.write('"""Map server package."""\n')

with open("tests/__init__.py", "w") as f:
    f.write("# tests package\n")

print("Project structure ready.")

Project structure ready.


In [4]:
%%writefile map_servers/params.py
from dataclasses import dataclass


@dataclass
class GeocodingServerParams:
    """
    Configuration for the Nominatim-based geocoding server.
    """
    base_url: str = "https://nominatim.openstreetmap.org"
    user_agent: str = "map-mcp-agent/0.1 (mohammad.m.mazeh@gmail.com)"
    timeout: int = 10


@dataclass
class RoutingServerParams:
    """
    Configuration for the OSRM-based routing server.
    """
    base_url: str = "https://router.project-osrm.org"
    profile: str = "driving"
    timeout: int = 10

Writing map_servers/params.py


In [5]:
%%writefile map_servers/geocoding_server.py
from typing import List, Dict, Any, Optional

import requests

from map_servers.params import GeocodingServerParams


def _create_session(params: GeocodingServerParams) -> requests.Session:
    """
    Build a configured HTTP session for talking to the Nominatim API.

    Nominatim's usage policy requires a descriptive User-Agent header
    that identifies your application.
    """
    session = requests.Session()
    session.headers.update(
        {
            "User-Agent": params.user_agent,
            "Accept": "application/json",
        }
    )
    return session


def geocode_address_raw(
    params: GeocodingServerParams, address: str, limit: int = 3
) -> List[Dict[str, Any]]:
    """
    Call the Nominatim /search endpoint to turn an address into coordinates.

    Returns the raw JSON list from the API.
    """
    session = _create_session(params)
    response = session.get(
        f"{params.base_url}/search",
        params={
            "q": address,
            "format": "jsonv2",
            "limit": limit,
            "addressdetails": 1,
        },
        timeout=params.timeout,
    )
    response.raise_for_status()
    return response.json()


def reverse_geocode_raw(
    params: GeocodingServerParams, lat: float, lon: float
) -> Dict[str, Any]:
    """
    Call the Nominatim /reverse endpoint to turn coordinates into an address.

    Returns the raw JSON object from the API.
    """
    session = _create_session(params)
    response = session.get(
        f"{params.base_url}/reverse",
        params={
            "lat": lat,
            "lon": lon,
            "format": "jsonv2",
            "addressdetails": 1,
        },
        timeout=params.timeout,
    )
    response.raise_for_status()
    return response.json()

_POI_KEYWORDS = {
    "coffee": "cafe",
    "coffee shop": "cafe",
    "coffee shops": "cafe",
    "espresso": "cafe",
}


def _normalize_poi_query(query: str) -> str:
    """
    Try to normalize generic POI phrases into more OSM-friendly keywords.

    Example: "coffee shop" -> "cafe".
    """
    q_lower = query.lower().strip()
    for key, replacement in _POI_KEYWORDS.items():
        if key in q_lower:
            return replacement
    return query.strip()


def search_poi_raw(
    params: GeocodingServerParams,
    query: str,
    city: Optional[str] = None,
    limit: int = 5,
) -> List[Dict[str, Any]]:
    """
    Look up points of interest (POIs) using the Nominatim /search endpoint.

    The function:
      * normalizes generic queries (coffee/coffee shop -> cafe),
      * tries multiple query patterns (e.g. "cafe Berlin", "cafe in Berlin"),
      * falls back to a generic search if needed.

    This makes queries like "coffee shops in Berlin" much more robust.
    """
    base_query = _normalize_poi_query(query)
    attempts: List[str] = []

    if city:
        attempts.append(f"{base_query} {city}")
        attempts.append(f"{base_query} in {city}")

    attempts.append(base_query)

    q_original = query.strip()
    if q_original and q_original != base_query:
        attempts.append(q_original)

    session = _create_session(params)

    last_exception: Optional[Exception] = None

    for q_text in attempts:
        if not q_text:
            continue

        request_params = {
            "q": q_text,
            "format": "jsonv2",
            "limit": limit,
            "addressdetails": 1,
        }

        if city and city.lower() == "berlin":
            request_params["countrycodes"] = "de"

        try:
            response = session.get(
                f"{params.base_url}/search",
                params=request_params,
                timeout=params.timeout,
            )
            response.raise_for_status()
            data = response.json()
        except Exception as exc:
            last_exception = exc
            continue

        if data:
            return data

    if last_exception is not None:
        raise last_exception

    return []


def simplify_geocode_results(
    results: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Convert raw Nominatim search results into a small, stable schema.
    """
    simplified: List[Dict[str, Any]] = []
    for item in results:
        try:
            lat = float(item["lat"])
            lon = float(item["lon"])
        except (KeyError, TypeError, ValueError):
            continue

        simplified.append(
            {
                "display_name": item.get("display_name"),
                "lat": lat,
                "lon": lon,
                "type": item.get("type"),
                "importance": item.get("importance"),
            }
        )
    return simplified


def simplify_reverse_result(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert a raw Nominatim reverse result into a small schema.
    """
    try:
        lat = float(data["lat"])
        lon = float(data["lon"])
    except (KeyError, TypeError, ValueError):
        lat = None
        lon = None

    return {
        "display_name": data.get("display_name"),
        "lat": lat,
        "lon": lon,
        "address": data.get("address", {}),
    }


def simplify_poi_results(
    results: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Convert raw POI search results into a small schema.

    This reuses the geocode simplification and adds basic category info.
    """
    simplified: List[Dict[str, Any]] = []
    base = simplify_geocode_results(results)
    for item, raw in zip(base, results):
        item["class"] = raw.get("class")
        item["category"] = raw.get("category")
        simplified.append(item)
    return simplified

Writing map_servers/geocoding_server.py


In [6]:
%%writefile map_servers/routing_server.py
from typing import List, Dict, Any, Tuple

import requests

from map_servers.params import RoutingServerParams


def _create_session(params: RoutingServerParams) -> requests.Session:
    """
    Build a configured HTTP session for talking to the OSRM API.
    """
    session = requests.Session()
    session.headers.update({"Accept": "application/json"})
    return session


def route_raw(
    params: RoutingServerParams,
    start_lat: float,
    start_lon: float,
    end_lat: float,
    end_lon: float,
) -> Dict[str, Any]:
    """
    Call OSRM's /route service to compute the fastest route between two points.

    OSRM expects coordinates as lon,lat in the URL path.
    """
    session = _create_session(params)
    coordinates = f"{start_lon},{start_lat};{end_lon},{end_lat}"
    url = f"{params.base_url}/route/v1/{params.profile}/{coordinates}"
    response = session.get(
        url,
        params={
            "overview": "false",
            "geometries": "geojson",
            "steps": "false",
        },
        timeout=params.timeout,
    )
    response.raise_for_status()
    return response.json()


def nearest_raw(
    params: RoutingServerParams,
    lat: float,
    lon: float,
) -> Dict[str, Any]:
    """
    Call OSRM's /nearest service to snap a point onto the road network.
    """
    session = _create_session(params)
    url = f"{params.base_url}/nearest/v1/{params.profile}/{lon},{lat}"
    response = session.get(
        url,
        params={"number": 1},
        timeout=params.timeout,
    )
    response.raise_for_status()
    return response.json()


def table_raw(
    params: RoutingServerParams,
    coordinates: List[Tuple[float, float]],
) -> Dict[str, Any]:
    """
    Call OSRM's /table service to get a travel time and distance matrix.

    `coordinates` is a list of (lat, lon) tuples.
    """
    if not coordinates:
        raise ValueError("coordinates list must not be empty")

    coord_str = ";".join(f"{lon},{lat}" for lat, lon in coordinates)
    session = _create_session(params)
    url = f"{params.base_url}/table/v1/{params.profile}/{coord_str}"
    response = session.get(
        url,
        params={"annotations": "duration,distance"},
        timeout=params.timeout,
    )
    response.raise_for_status()
    return response.json()


def _seconds_to_minutes(value: float) -> float:
    """
    Convert seconds to minutes, rounded to two decimals.
    """
    return round(value / 60.0, 2)


def simplify_route(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert a raw OSRM /route response into a simple summary.

    Adds both duration in seconds and duration in minutes.
    """
    if data.get("code") != "Ok":
        raise ValueError(f"OSRM route error: {data.get('code')}")

    routes = data.get("routes") or []
    if not routes:
        raise ValueError("OSRM route response contained no routes")

    route = routes[0]
    waypoints = data.get("waypoints") or []

    duration_s = route.get("duration")
    distance_m = route.get("distance")

    summary: Dict[str, Any] = {
        "distance_m": distance_m,
        "duration_s": duration_s,
        "duration_min": _seconds_to_minutes(duration_s) if duration_s is not None else None,
    }

    if len(waypoints) >= 1:
        summary.setdefault("summary", {})
        summary["summary"]["start"] = waypoints[0].get("name")

    if len(waypoints) >= 2:
        summary.setdefault("summary", {})
        summary["summary"]["end"] = waypoints[-1].get("name")

    return summary


def simplify_nearest(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert a raw OSRM /nearest response into a small schema.
    """
    waypoints = data.get("waypoints") or []
    if not waypoints:
        raise ValueError("OSRM nearest response contained no waypoints")

    wp = waypoints[0]
    location = wp.get("location") or [None, None]
    lon, lat = location[0], location[1]

    return {
        "name": wp.get("name"),
        "lat": lat,
        "lon": lon,
        "distance_m": wp.get("distance"),
    }


def simplify_table(data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert a raw OSRM /table response into a small schema.

    Includes both duration in seconds and an extra matrix with duration in minutes.
    """
    durations_s = data.get("durations")
    distances_m = data.get("distances")

    durations_min = None
    if durations_s is not None:
        durations_min = [
            [_seconds_to_minutes(val) if val is not None else None for val in row]
            for row in durations_s
        ]

    return {
        "durations_s": durations_s,
        "durations_min": durations_min,
        "distances_m": distances_m,
    }

Writing map_servers/routing_server.py


In [7]:
%%writefile agent_app.py
from typing import List, Optional

from typing_extensions import TypedDict

from agents import Agent, Runner, ModelSettings, function_tool

from map_servers.params import GeocodingServerParams, RoutingServerParams
from map_servers.geocoding_server import (
    geocode_address_raw,
    reverse_geocode_raw,
    search_poi_raw,
    simplify_geocode_results,
    simplify_reverse_result,
    simplify_poi_results,
)
from map_servers.routing_server import (
    route_raw,
    nearest_raw,
    table_raw,
    simplify_route,
    simplify_nearest,
    simplify_table,
)

GEOCODING_SERVER = GeocodingServerParams()
ROUTING_SERVER = RoutingServerParams()


@function_tool
def geocode_address(address: str, limit: int = 3) -> dict:
    """
    Geocode a free-form address string into coordinates.

    Args:
        address: The address or place name to search for.
        limit: Maximum number of results to return.
    """
    print(f"[TOOL] geocode_address(address={address!r}, limit={limit})")
    raw_results = geocode_address_raw(GEOCODING_SERVER, address, limit=limit)
    simplified = simplify_geocode_results(raw_results)
    print(f"[TOOL] geocode_address -> {len(simplified)} result(s)")
    return {"results": simplified}


@function_tool
def reverse_geocode(lat: float, lon: float) -> dict:
    """
    Reverse geocode coordinates into a human-readable address.

    Args:
        lat: Latitude in decimal degrees.
        lon: Longitude in decimal degrees.
    """
    print(f"[TOOL] reverse_geocode(lat={lat}, lon={lon})")
    raw_result = reverse_geocode_raw(GEOCODING_SERVER, lat=lat, lon=lon)
    simplified = simplify_reverse_result(raw_result)
    print(f"[TOOL] reverse_geocode -> {simplified.get('display_name')!r}")
    return simplified


@function_tool
def search_poi(query: str, city: Optional[str] = None, limit: int = 5) -> dict:
    """
    Search for points of interest (POIs) such as cafes or museums.

    Args:
        query: What you are looking for (e.g. 'coffee shop', 'museum').
        city: Optional city name to narrow the search.
        limit: Maximum number of results to return.
    """
    print(f"[TOOL] search_poi(query={query!r}, city={city!r}, limit={limit})")
    raw_results = search_poi_raw(GEOCODING_SERVER, query=query, city=city, limit=limit)
    simplified = simplify_poi_results(raw_results)
    print(f"[TOOL] search_poi -> {len(simplified)} result(s)")
    return {"results": simplified}


class Coordinate(TypedDict):
    lat: float
    lon: float


@function_tool
def route_between_points(
    start: Coordinate,
    end: Coordinate,
) -> dict:
    """
    Compute a car route between two coordinates using OSRM.

    Args:
        start: Starting coordinate as {"lat": ..., "lon": ...}.
        end: Destination coordinate as {"lat": ..., "lon": ...}.
    """
    print(f"[TOOL] route_between_points(start={start}, end={end})")
    raw = route_raw(
        ROUTING_SERVER,
        start_lat=start["lat"],
        start_lon=start["lon"],
        end_lat=end["lat"],
        end_lon=end["lon"],
    )
    simplified = simplify_route(raw)
    print(
        f"[TOOL] route_between_points -> distance={simplified['distance_m']} m, "
        f"duration={simplified['duration_s']} s (~{simplified['duration_min']} min)"
    )
    return simplified


@function_tool
def nearest_road(location: Coordinate) -> dict:
    """
    Snap a coordinate onto the nearest road using OSRM.

    Args:
        location: Coordinate to snap as {"lat": ..., "lon": ...}.
    """
    print(f"[TOOL] nearest_road(location={location})")
    raw = nearest_raw(ROUTING_SERVER, lat=location["lat"], lon=location["lon"])
    simplified = simplify_nearest(raw)
    print(
        f"[TOOL] nearest_road -> name={simplified['name']!r}, "
        f"lat={simplified['lat']}, lon={simplified['lon']}"
    )
    return simplified


@function_tool
def distance_matrix(locations: List[Coordinate]) -> dict:
    """
    Build a travel-time and distance matrix between several coordinates.

    Args:
        locations: List of coordinates as {"lat": ..., "lon": ...}.
    """
    print(f"[TOOL] distance_matrix(locations={locations})")
    if len(locations) < 2:
        raise ValueError("Provide at least two locations for a matrix")

    coords = [(loc["lat"], loc["lon"]) for loc in locations]
    raw = table_raw(ROUTING_SERVER, coords)
    simplified = simplify_table(raw)

    durations_s = simplified.get("durations_s")
    durations_min = simplified.get("durations_min")
    if durations_s is not None:
        print(
            f"[TOOL] distance_matrix -> "
            f"{len(durations_s)}x{len(durations_s[0])} matrix "
            f"(seconds and minutes)"
        )
    return simplified


map_agent = Agent(
    name="MapAssistant",
    instructions=(
        "You are a mapping assistant. You can geocode addresses, "
        "reverse geocode coordinates, find nearby points of interest, "
        "and compute routes and distance matrices. "
        "Tool outputs include durations in seconds and in minutes; "
        "when you talk about time, prefer the minutes fields and do not "
        "recompute them yourself. "
        "Use the tools whenever you need real geographic data, "
        "then explain results in simple language."
    ),
    model="gpt-4.1-mini",
    tools=[
        geocode_address,
        reverse_geocode,
        search_poi,
        route_between_points,
        nearest_road,
        distance_matrix,
    ],
    model_settings=ModelSettings(tool_choice="auto"),
)


async def run_map_agent_async(user_input: str) -> str:
    """
    Async helper for running the map agent.

    This uses Runner.run(), which is async and works inside notebooks.
    """
    print(f"[AGENT] User input: {user_input!r}")
    result = await Runner.run(map_agent, user_input)
    print(f"[AGENT] Final output: {result.final_output!r}")
    return str(result.final_output)


if __name__ == "__main__":
    import asyncio
    import sys

    query = " ".join(sys.argv[1:]) or "Give me a short demo of your mapping abilities."

    async def _main():
        print(await run_map_agent_async(query))

    asyncio.run(_main())

Writing agent_app.py


In [8]:
%%writefile tests/test_parsing.py
import unittest

from map_servers.geocoding_server import (
    simplify_geocode_results,
    simplify_reverse_result,
    simplify_poi_results,
)
from map_servers.routing_server import (
    simplify_route,
    simplify_table,
    simplify_nearest,
)


class TestGeocodingParsing(unittest.TestCase):
    def test_simplify_geocode_results_basic(self):
        sample = [
            {
                "display_name": "Eiffel Tower, Paris, Île-de-France, France",
                "lat": "48.8582602",
                "lon": "2.2944991",
                "type": "attraction",
                "importance": 0.9,
            }
        ]
        simplified = simplify_geocode_results(sample)
        self.assertEqual(len(simplified), 1)
        self.assertIn("lat", simplified[0])
        self.assertIn("lon", simplified[0])
        self.assertAlmostEqual(simplified[0]["lat"], 48.8582602, places=4)
        self.assertEqual(simplified[0]["type"], "attraction")

    def test_simplify_geocode_results_skips_malformed(self):
        sample = [
            {
                "display_name": "Good",
                "lat": "48.0",
                "lon": "2.0",
            },
            {
                "display_name": "Bad",
                "type": "something",
            },
        ]
        simplified = simplify_geocode_results(sample)
        self.assertEqual(len(simplified), 1)
        self.assertEqual(simplified[0]["display_name"], "Good")

    def test_simplify_reverse_result_invalid_coords(self):
        sample = {
            "display_name": "Somewhere",
            "lat": "not-a-number",
            "lon": "not-a-number",
            "address": {"city": "Nowhere"},
        }
        simplified = simplify_reverse_result(sample)
        self.assertIsNone(simplified["lat"])
        self.assertIsNone(simplified["lon"])
        self.assertEqual(simplified["address"]["city"], "Nowhere")


class TestPOIParsing(unittest.TestCase):
    def test_simplify_poi_results_adds_class_category(self):
        raw = [
            {
                "display_name": "Nice Cafe, Berlin, Germany",
                "lat": "52.520008",
                "lon": "13.404954",
                "type": "cafe",
                "importance": 0.7,
                "class": "amenity",
                "category": "cafe",
            }
        ]
        simplified = simplify_poi_results(raw)
        self.assertEqual(len(simplified), 1)
        item = simplified[0]
        self.assertEqual(item["class"], "amenity")
        self.assertEqual(item["category"], "cafe")
        self.assertAlmostEqual(item["lat"], 52.520008, places=4)


class TestRoutingParsing(unittest.TestCase):
    def test_simplify_route_ok(self):
        sample = {
            "code": "Ok",
            "routes": [
                {"distance": 1000.0, "duration": 300.0}
            ],
            "waypoints": [
                {"name": "Start"},
                {"name": "End"},
            ],
        }
        simplified = simplify_route(sample)
        self.assertAlmostEqual(simplified["distance_m"], 1000.0)
        self.assertAlmostEqual(simplified["duration_s"], 300.0)
        self.assertAlmostEqual(simplified["duration_min"], 5.0)
        self.assertEqual(simplified["summary"]["start"], "Start")
        self.assertEqual(simplified["summary"]["end"], "End")

    def test_simplify_route_non_ok_raises(self):
        sample = {
            "code": "NoRoute",
            "routes": [],
            "waypoints": [],
        }
        with self.assertRaises(ValueError):
            simplify_route(sample)

    def test_simplify_table(self):
        sample = {
            "durations": [
                [0.0, 100.0],
                [100.0, 0.0],
            ],
            "distances": [
                [0.0, 500.0],
                [500.0, 0.0],
            ],
        }
        simplified = simplify_table(sample)

        self.assertIn("durations_s", simplified)
        self.assertEqual(len(simplified["durations_s"]), 2)
        self.assertEqual(simplified["durations_s"][0][1], 100.0)
        self.assertEqual(simplified["distances_m"][0][1], 500.0)

        self.assertIn("durations_min", simplified)
        self.assertAlmostEqual(simplified["durations_min"][0][1], 100.0 / 60.0, places=2)

    def test_simplify_nearest(self):
        sample = {
            "waypoints": [
                {
                    "name": "Random Street",
                    "distance": 3.5,
                    "location": [2.2944991, 48.8582602],
                }
            ]
        }
        simplified = simplify_nearest(sample)
        self.assertEqual(simplified["name"], "Random Street")
        self.assertAlmostEqual(simplified["lat"], 48.8582602, places=4)
        self.assertAlmostEqual(simplified["lon"], 2.2944991, places=4)
        self.assertAlmostEqual(simplified["distance_m"], 3.5, places=2)


if __name__ == "__main__":
    unittest.main()

Writing tests/test_parsing.py


In [9]:
%%writefile tests/test_live_services.py
import os
import time
import unittest

from map_servers.params import GeocodingServerParams, RoutingServerParams
from map_servers.geocoding_server import (
    geocode_address_raw,
    search_poi_raw,
    simplify_geocode_results,
    simplify_poi_results,
)
from map_servers.routing_server import (
    route_raw,
    simplify_route,
)


RUN_LIVE = os.environ.get("RUN_LIVE_TESTS", "0") == "1"


@unittest.skipUnless(RUN_LIVE, "Set RUN_LIVE_TESTS=1 to run live API tests")
class TestLiveMapServices(unittest.TestCase):
    """
    These tests hit the real Nominatim and OSRM public endpoints.

    They show that the two map servers (geocoding and routing) are
    actually working end-to-end, not just assumed.
    """

    @classmethod
    def setUpClass(cls):
        cls.geo_params = GeocodingServerParams()
        cls.route_params = RoutingServerParams()

    def test_live_geocode_eiffel(self):
        raw = geocode_address_raw(
            self.geo_params,
            "Eiffel Tower, Paris",
            limit=1,
        )
        simplified = simplify_geocode_results(raw)
        self.assertGreaterEqual(len(simplified), 1)
        lat = simplified[0]["lat"]
        lon = simplified[0]["lon"]
        self.assertTrue(48.85 < lat < 48.87)
        self.assertTrue(2.28 < lon < 2.31)
        time.sleep(1)

    def test_live_route_eiffel_to_louvre(self):
        raw = route_raw(
            self.route_params,
            start_lat=48.85826,
            start_lon=2.2945,
            end_lat=48.8606,
            end_lon=2.3376,
        )
        summary = simplify_route(raw)
        distance = summary["distance_m"]
        self.assertTrue(2000 <= distance <= 7000)
        time.sleep(1)

    def test_live_poi_cafe_berlin(self):
        raw = search_poi_raw(
            self.geo_params,
            query="cafe",
            city="Berlin",
            limit=5,
        )
        simplified = simplify_poi_results(raw)
        self.assertGreaterEqual(len(simplified), 1)
        first = simplified[0]
        self.assertTrue(-90.0 <= first["lat"] <= 90.0)
        self.assertTrue(-180.0 <= first["lon"] <= 180.0)
        time.sleep(1)


if __name__ == "__main__":
    unittest.main()

Writing tests/test_live_services.py


In [10]:
import os
import unittest
import sys
from pathlib import Path

os.environ["RUN_LIVE_TESTS"] = "1"

root = Path(".").resolve()
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print("Running tests from:", root)
print("RUN_LIVE_TESTS =", os.environ.get("RUN_LIVE_TESTS"))

suite = unittest.defaultTestLoader.discover("tests")
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_live_geocode_eiffel (test_live_services.TestLiveMapServices.test_live_geocode_eiffel) ... 

Running tests from: /content
RUN_LIVE_TESTS = 1


ok
test_live_poi_cafe_berlin (test_live_services.TestLiveMapServices.test_live_poi_cafe_berlin) ... ok
test_live_route_eiffel_to_louvre (test_live_services.TestLiveMapServices.test_live_route_eiffel_to_louvre) ... ok
test_simplify_geocode_results_basic (test_parsing.TestGeocodingParsing.test_simplify_geocode_results_basic) ... ok
test_simplify_geocode_results_skips_malformed (test_parsing.TestGeocodingParsing.test_simplify_geocode_results_skips_malformed) ... ok
test_simplify_reverse_result_invalid_coords (test_parsing.TestGeocodingParsing.test_simplify_reverse_result_invalid_coords) ... ok
test_simplify_poi_results_adds_class_category (test_parsing.TestPOIParsing.test_simplify_poi_results_adds_class_category) ... ok
test_simplify_nearest (test_parsing.TestRoutingParsing.test_simplify_nearest) ... ok
test_simplify_route_non_ok_raises (test_parsing.TestRoutingParsing.test_simplify_route_non_ok_raises) ... ok
test_simplify_route_ok (test_parsing.TestRoutingParsing.test_simplify_route_ok)

<unittest.runner.TextTestResult run=11 errors=0 failures=0>

In [12]:
from map_servers.params import GeocodingServerParams, RoutingServerParams
from map_servers.geocoding_server import (
    geocode_address_raw,
    search_poi_raw,
    simplify_geocode_results,
    simplify_poi_results,
)
from map_servers.routing_server import (
    route_raw,
    simplify_route,
    nearest_raw,
    simplify_nearest,
    table_raw,
    simplify_table,
)
from agent_app import run_map_agent_async


async def demo():
    geo_params = GeocodingServerParams()
    route_params = RoutingServerParams()

    print("=== DIRECT SERVER DEMO ===")

    ts_raw = geocode_address_raw(geo_params, "Times Square, New York City", limit=1)
    cp_raw = geocode_address_raw(geo_params, "Central Park, New York City", limit=1)
    times_square = simplify_geocode_results(ts_raw)[0]
    central_park = simplify_geocode_results(cp_raw)[0]

    print("Times Square:", times_square)
    print("Central Park:", central_park)

    route_data = route_raw(
        route_params,
        start_lat=times_square["lat"],
        start_lon=times_square["lon"],
        end_lat=central_park["lat"],
        end_lon=central_park["lon"],
    )
    route_summary = simplify_route(route_data)
    print("\nRoute summary (Times Square -> Central Park):", route_summary)
    if route_summary["distance_m"] is not None:
        print(
            f"  Distance: {route_summary['distance_m']} m "
            f"({route_summary['distance_m'] / 1000:.2f} km)"
        )
    if route_summary["duration_s"] is not None:
        print(
            f"  Duration: {route_summary['duration_s']} s "
            f"({route_summary['duration_min']} min)"
        )

    poi_raw = search_poi_raw(
        geo_params, query="sushi restaurant", city="Tokyo", limit=5
    )
    poi_simplified = simplify_poi_results(poi_raw)
    print("\nSample POIs (sushi restaurants in Tokyo):")
    for item in poi_simplified:
        print(" -", item["display_name"], "| lat:", item["lat"], "lon:", item["lon"])

    nearest_data = nearest_raw(
        route_params, lat=times_square["lat"], lon=times_square["lon"]
    )
    nearest_summary = simplify_nearest(nearest_data)
    print("\nNearest road to Times Square:", nearest_summary)

    bb_raw = geocode_address_raw(geo_params, "Big Ben, London", limit=1)
    bp_raw = geocode_address_raw(geo_params, "Buckingham Palace, London", limit=1)
    tb_raw = geocode_address_raw(geo_params, "Tower Bridge, London", limit=1)

    big_ben = simplify_geocode_results(bb_raw)[0]
    buckingham = simplify_geocode_results(bp_raw)[0]
    tower_bridge = simplify_geocode_results(tb_raw)[0]

    coords = [
        (big_ben["lat"], big_ben["lon"]),
        (buckingham["lat"], buckingham["lon"]),
        (tower_bridge["lat"], tower_bridge["lon"]),
    ]
    matrix_raw = table_raw(route_params, coords)
    matrix_summary = simplify_table(matrix_raw)
    print("\nDistance matrix durations (seconds):")
    for row in matrix_summary["durations_s"]:
        print(" ", row)
    print("\nDistance matrix durations (minutes):")
    for row in matrix_summary["durations_min"]:
        print(" ", row)

    print("\n=== AGENT DEMO ===")

    query1 = (
        "Find a driving route from Times Square to Central Park in New York City. "
        "Tell me the distance and approximate driving time, using the tools."
    )
    result1 = await run_map_agent_async(query1)
    print("\nAgent result 1:\n", result1)

    query2 = (
        "In Tokyo, find a few sushi restaurants and give me their coordinates. "
        "Use the POI search tool rather than guessing."
    )
    result2 = await run_map_agent_async(query2)
    print("\nAgent result 2:\n", result2)

    query3 = (
        "Given Big Ben, Buckingham Palace, and Tower Bridge in London, "
        "use the distance matrix tool to compute travel times, then explain "
        "which pair of landmarks is closest in terms of travel time. "
        "Use the durations in minutes returned by the tools."
    )
    result3 = await run_map_agent_async(query3)
    print("\nAgent result 3:\n", result3)


await demo()

=== DIRECT SERVER DEMO ===
Times Square: {'display_name': 'Times Square, Theater District, Manhattan Community Board 5, Manhattan, New York County, City of New York, New York, 10036, United States of America', 'lat': 40.7570095, 'lon': -73.9859724, 'type': 'pedestrian', 'importance': 0.5936829858927538}
Central Park: {'display_name': 'Central Park, New York County, City of New York, New York, United States of America', 'lat': 40.7827725, 'lon': -73.9653627, 'type': 'park', 'importance': 0.6147697757847301}

Route summary (Times Square -> Central Park): {'distance_m': 4990, 'duration_s': 548.1, 'duration_min': 9.13, 'summary': {'start': '7th Avenue', 'end': ''}}
  Distance: 4990 m (4.99 km)
  Duration: 548.1 s (9.13 min)

Sample POIs (sushi restaurants in Tokyo):
 - Sushi, まいろーど四つ木商店街, 四つ木一丁目, 四つ木, 葛飾区, 東京都, 124-0011, 日本 | lat: 35.7355801 lon: 139.8375226

Nearest road to Times Square: {'name': '7th Avenue', 'lat': 40.757096, 'lon': -73.986185, 'distance_m': 20.36038687}

Distance matri